<a href="https://colab.research.google.com/github/Ehsan-data/JAVA/blob/master/CIFAR_10_ResNet(10_epochs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
#checks for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters
num_epochs = 10
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

#  dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

# convolution
def conv_res(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Res block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv_res(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv_res(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet class
class ResNet(nn.Module):
    def __init__(self, block, layer_arr, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv_res(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.set_layer(block, 16, layer_arr[0])
        self.layer2 = self.set_layer(block, 32, layer_arr[1], 2)
        self.layer3 = self.set_layer(block, 64, layer_arr[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def set_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv_res(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layer_arr = []
        layer_arr.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layer_arr.append(block(out_channels, out_channels))
        return nn.Sequential(*layer_arr)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# learning rate update
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# model training
tot_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (img, classes) in enumerate(train_loader):
        img = img.to(device)
        classes = classes.to(device)

        # Forward pass
        outputs = model(img)
        loss = criterion(outputs, classes)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, tot_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# model evaluation
model.eval()
with torch.no_grad():
    correct_pred = 0
    tot = 0
    for img, classes in test_loader:
        img = img.to(device)
        classes = classes.to(device)
        outputs = model(img)
        _, pred = torch.max(outputs.data, 1)
        tot += classes.size(0)
        correct_pred += (pred == classes).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct_pred / tot))

# Saveing the checkpoints
torch.save(model.state_dict(), 'resnet.ckpt')



Files already downloaded and verified
Epoch [1/10], Step [100/500] Loss: 1.5296
Epoch [1/10], Step [200/500] Loss: 1.4502
Epoch [1/10], Step [300/500] Loss: 1.1180
Epoch [1/10], Step [400/500] Loss: 1.1509
Epoch [1/10], Step [500/500] Loss: 1.0823
Epoch [2/10], Step [100/500] Loss: 0.9899
Epoch [2/10], Step [200/500] Loss: 0.8545
Epoch [2/10], Step [300/500] Loss: 0.9672
Epoch [2/10], Step [400/500] Loss: 0.8611
Epoch [2/10], Step [500/500] Loss: 1.1886
Epoch [3/10], Step [100/500] Loss: 1.0470
Epoch [3/10], Step [200/500] Loss: 1.0025
Epoch [3/10], Step [300/500] Loss: 0.8506
Epoch [3/10], Step [400/500] Loss: 0.8678
Epoch [3/10], Step [500/500] Loss: 0.6630
Epoch [4/10], Step [100/500] Loss: 0.6731
Epoch [4/10], Step [200/500] Loss: 0.8746
Epoch [4/10], Step [300/500] Loss: 0.7173
Epoch [4/10], Step [400/500] Loss: 0.7524
Epoch [4/10], Step [500/500] Loss: 0.6104
Epoch [5/10], Step [100/500] Loss: 0.8467
Epoch [5/10], Step [200/500] Loss: 0.6617
Epoch [5/10], Step [300/500] Loss: 0.5

In [ ]:
### most of the code adopted from                              
### https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
